In [5]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

c:\Users\cm-sa\.conda\envs\tf2.3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:00<00:00, 44.00it/s]


In [6]:
books = books["train"].train_test_split(test_size=0.2)

In [7]:
books["train"][0]

{'id': '83257',
 'translation': {'en': "Carried away by his misery, dazed by surprise, Julien was weak enough to say to her, in the tenderest of tones and one that sprang from the heart: 'Then, you no longer love me?'",
  'fr': 'Emporté par son malheur, égaré par la surprise, Julien eut la faiblesse de lui dire, du ton le plus tendre et qui venait de l’âme : Ainsi, vous ne m’aimez plus ?'}}

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

c:\Users\cm-sa\.conda\envs\tf2.3\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_books = books.map(preprocess_function, batched=True)

100%|██████████| 26/26 [00:02<00:00, 11.43ba/s]


In [11]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [12]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [17]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [15]:
tf_train_set = tokenized_books["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = tokenized_books["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [18]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3


ResourceExhaustedError: Graph execution error:

Detected at node 'tft5_for_conditional_generation/decoder/block_._4/layer_._2/DenseReluDense/wi/Tensordot/MatMul' defined at (most recent call last):
    File "c:\Users\cm-sa\.conda\envs\tf2.3\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\cm-sa\.conda\envs\tf2.3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\cm-sa\.conda\envs\tf2.3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\cm-sa\.conda\envs\tf2.3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\cm-sa\.conda\envs\tf2.3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\cm-sa\.conda\envs\tf2.3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_fo